# US City Dot Map (Geocode + Plot)

This notebook:
1. Cleans your city strings
2. Geocodes to latitude/longitude using OpenStreetMap (Nominatim)
3. Plots a dot map over the United States

**Notes**
- Nominatim is rate-limited. This notebook uses a delay to be polite.
- If some metro-area strings fail, it falls back to the first city in `A/B/C, ST`.


In [1]:
import re
import time
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from preprocess.data_utils import get_unique_cities
import plotly.express as px
from geopy.geocoders import GoogleV3
import os
os.chdir("..")  # or "../.." to go to project root


## 1) Paste your cities here

Replace the list below with your column values.


In [2]:
cities = get_unique_cities()
assert len(cities) > 0, "Please paste your city strings into the `cities` list."

## 2) Cleaning helpers


In [3]:
def normalize_city(s: str) -> str:
    """Remove extra tags/spaces that hurt geocoding."""
    s = re.sub(r"\s*\(Metropolitan Area\)\s*", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def fallback_first_city(s: str) -> str:
    """If we have 'A/B/C, ST', fall back to 'A, ST'."""
    if "/" in s and "," in s:
        left, st = s.split(",", 1)
        first = left.split("/")[0].strip()
        return f"{first}, {st.strip()}"
    return s


## 3) Geocode to lat/lon caching

This uses Geocodingapi. Please keep the delay (or increase it) to avoid being blocked.


In [ ]:
# import os
# import pandas as pd
# from geopy.geocoders import GoogleV3

# GOOGLE_API_KEY = ""
# geolocator = GoogleV3(api_key=GOOGLE_API_KEY)

# cache_path = "./data/geocoded_cities.csv"

# # Load cache if it exists
# if os.path.exists(cache_path):
#     cache_df = pd.read_csv(cache_path)
# else:
#     cache_df = pd.DataFrame(columns=["city_raw", "city_clean", "geocode_query", "lat", "lon"])

# # Make a dict for fast lookup
# cache_dict = {row["city_raw"]: row for _, row in cache_df.iterrows()}

# rows = []
# for raw in cities:
#     if raw in cache_dict and pd.notna(cache_dict[raw]["lat"]) and pd.notna(cache_dict[raw]["lon"]):
#         # Use cached result
#         row = cache_dict[raw]
#         rows.append({
#             "city_raw": raw,
#             "city_clean": row["city_clean"],
#             "geocode_query": row["geocode_query"],
#             "lat": row["lat"],
#             "lon": row["lon"],
#         })
#         continue

#     cleaned = normalize_city(raw)
#     q1 = f"{cleaned}, USA"
#     try:
#         loc = geolocator.geocode(q1)
#     except Exception as e:
#         print(f"Error geocoding {q1}: {e}")
#         loc = None

#     used_query = q1
#     if loc is None:
#         cleaned2 = fallback_first_city(cleaned)
#         q2 = f"{cleaned2}, USA"
#         try:
#             loc = geolocator.geocode(q2)
#         except Exception as e:
#             print(f"Error geocoding {q2}: {e}")
#             loc = None
#         used_query = q2

#     rows.append({
#         "city_raw": raw,
#         "city_clean": cleaned,
#         "geocode_query": used_query,
#         "lat": None if loc is None else loc.latitude,
#         "lon": None if loc is None else loc.longitude,
#     })

# # Save updated cache
# df_geo = pd.DataFrame(rows)
# # Combine with previous cache and drop duplicates
# df_geo = pd.concat([cache_df, df_geo]).drop_duplicates(subset=["city_raw"], keep="last")
# df_geo.to_csv(cache_path, index=False)
# print("Saved:", cache_path)

# df_geo.head()

Saved: ./data/geocoded_cities.csv


/var/folders/rt/_b9r914s0q7bcxbhbvg5mqfw0000gn/T/ipykernel_7148/2344886292.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_geo = pd.concat([cache_df, df_geo]).drop_duplicates(subset=["city_raw"], keep="last")


,city_raw,city_clean,geocode_query,lat,lon
0,"Milwaukee, WI","Milwaukee, WI","Milwaukee, WI, USA",43.041034,-87.909678
1,"Myrtle Beach, SC","Myrtle Beach, SC","Myrtle Beach, SC, USA",33.695422,-78.880209
2,"Nantucket, MA","Nantucket, MA","Nantucket, MA, USA",41.285325,-70.098800
3,"Punta Gorda, FL","Punta Gorda, FL","Punta Gorda, FL, USA",26.929784,-82.045366
4,"Little Rock, AR","Little Rock, AR","Little Rock, AR, USA",34.742829,-92.276290


In [9]:
cache_path = "./data/geocoded_cities.csv"  # or the path you used to save
df_geo = pd.read_csv(cache_path)
# Show failures (if any)
fail = df_geo[df_geo["lat"].isna()][["city_raw", "geocode_query"]]
print(f"Failed geocodes: {len(fail)}")
fail.head(25)

Failed geocodes: 0


,city_raw,geocode_query


## 4) Plot dot map over the USA


In [4]:
cache_path = "./data/geocoded_cities.csv"  # or the path you used to save
df_geo = pd.read_csv(cache_path)
df_plot = df_geo.dropna(subset=["lat", "lon"]).copy()

fig = px.scatter_geo(
    df_plot,
    lat="lat",
    lon="lon",
    hover_name="city_raw",
    scope="usa",
    projection="albers usa",
)
fig.update_traces(marker=dict(size=6))
fig.update_layout(title="US City Dot Map")
fig.show()

## 5) Optional: characterize cities by size/color

If you have a metric per city (e.g., counts, score), merge it into `df_geo` and use `size=` / `color=`.


In [ ]:
# Example (uncomment and adapt):
# city_metric = pd.DataFrame({
#     "city_raw": ["Albany, NY", "Austin, TX"],
#     "count": [12, 30],
#     "category": ["A", "B"],
# })
# df2 = df_geo.merge(city_metric, on="city_raw", how="left")
# df2 = df2.dropna(subset=["lat", "lon"])
# fig = px.scatter_geo(
#     df2,
#     lat="lat", lon="lon",
#     hover_name="city_raw",
#     size="count",
#     color="category",
#     scope="usa",
#     projection="albers usa",
# )
# fig.show()

In [ ]:
# Save geocoded results for reuse
out_path = "geocoded_cities.csv"
df_geo.to_csv(out_path, index=False)
print("Saved:", out_path)